In [1]:
# The code was removed by Watson Studio for sharing.

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190903070647-0000
KERNEL_ID = 49492b07-a361-4d7a-ade0-ef8355c2a218


In [2]:
df.createOrReplaceTempView('df')
df_energy = spark.sql("""
select sqrt(sum(x*x)+sum(y*y)+sum(z*z)) as label, class from df group by class
""")      
df_energy.createOrReplaceTempView('df_energy')          

In [3]:
df_energy.show()

+------------------+--------------+
|             label|         class|
+------------------+--------------+
| 8959.680239829991| Use_telephone|
| 9737.511232342687| Standup_chair|
| 12542.96539897962|      Eat_meat|
|13225.945637269193|     Getup_bed|
|15003.269043778426|   Drink_glass|
|14454.885091207056|    Pour_water|
|10616.408809008817|     Comb_hair|
|11082.626493751379|  Climb_stairs|
|10261.338314274606| Sitdown_chair|
|6783.4063714331605|   Liedown_bed|
| 7173.493500380411|Descend_stairs|
| 11785.39634462923|   Brush_teeth|
| 6071.460120926432|      Eat_soup|
+------------------+--------------+



In [4]:
df_join = spark.sql('select * from df inner join df_energy on df.class=df_energy.class')

In [5]:
df_join.show()

+---+---+---+--------------------+------------+------------------+------------+
|  x|  y|  z|              source|       class|             label|       class|
+---+---+---+--------------------+------------+------------------+------------+
| 30| 36| 52|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 30| 36| 32|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 32| 33| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 30| 37| 50|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 31| 37| 50|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 29| 38| 50|Accelerometer-201...|Climb_

In [6]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer


vectorAssembler = VectorAssembler(inputCols=["x","y","z"], outputCol="features")
normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)

In [7]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [8]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, normalizer,lr])

In [9]:
model = pipeline.fit(df_join)

In [10]:
prediction = model.transform(df_join)

In [11]:
prediction.show()

+---+---+---+--------------------+------------+------------------+------------+----------------+--------------------+------------------+
|  x|  y|  z|              source|       class|             label|       class|        features|       features_norm|        prediction|
+---+---+---+--------------------+------------+------------------+------------+----------------+--------------------+------------------+
| 30| 36| 52|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|[30.0,36.0,52.0]|[0.25423728813559...|12378.337524478058|
| 30| 36| 32|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|[30.0,36.0,32.0]|[0.30612244897959...|10949.223755865942|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|[32.0,30.0,36.0]|[0.32653061224489...|11232.049623881609|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|[32.0,30.0,36.0]|[0.32653061224489...|11232.049623881609|
| 32| 30| 36|Accelerometer-201...|Climb_s

In [8]:
model.stages[2].summary.r2


0.0912330479847846